In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import date
import requests
import psycopg2
import time

In [3]:
# 셀리니움 크롤링
url = 'C:/Users/h/Desktop/chatbot/'
driver = webdriver.Chrome(url + 'chromedriver.exe')
driver.get("https://safecity.seoul.go.kr/acdnt/sbwyIndex.do")
time.sleep(3)

C:\Users\h\AppData\Local\Temp\ipykernel_11040\1521152426.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(url + 'chromedriver.exe')


In [2]:
# 셀레니움 창오픈 없이 실행

PATH = 'C:/Users/h/Desktop/chatbot/'
options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(PATH + 'chromedriver.exe', options=options)


C:\Users\h\AppData\Local\Temp\ipykernel_27392\1918150743.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH + 'chromedriver.exe', options=options)


In [3]:
driver.get("https://safecity.seoul.go.kr/acdnt/sbwyIndex.do")

In [4]:
# 데이터 긁어오기
parentElement = driver.find_elements(By.XPATH, '//*[@id="dv_as_timeline"]/li')

subli=[]

# ul 태그 아래 있는 li 반복 뽑기
for i in parentElement:
    i.click()
    time.sleep(0.05)
    a = i.text
    subli.append(a)
    print(a)
    print(type(a))
    print()
    
    
    i.click()

    

지하철사고
2022-11-24 06:00
서울교통공사에서 알려드립니다. 금일 08시부터 4호선에서「전국장애인차별철폐연대」의 ‘장애인 권리예산 확보’를 위한 출근길 지하철 타기 선전전이 예정되어 있습니다. 이로 인해 4호선 해당구간 열차운행이 상당시간 지연될 수 있으니 이점 참고하여 열차를 이용해 주시기 바랍니다.
<class 'str'>



In [ ]:
#데이터베이스 새로 만들기 (복합 키 세팅)
# create table
def db_create():
    cur.execute("CREATE TABLE IF NOT EXISTS subaccdata(acctime VARCHAR(30) NOT NULL, content VARCHAR(300) NOT NULL, primary key(acctime, content));")
    conn.commit()

db_create()

In [6]:
# db연결
conn = psycopg2.connect(host="ec2-23-21-207-93.compute-1.amazonaws.com", 
                        dbname="d3oubpekvnbupv", 
                        user="grxhirqndvyqvv", 
                        password="6f1afaafe16d245c70666bdb8c831aa876e62b380a1544f5dc832c51f27cece6", 
                        port="5432")

# 데이터 조작 객체 생성
cur = conn.cursor()

In [7]:
#new db insert
for i in subli:
    # slice 후 첫번째 요소 제거 tuple화
    subdb = i.split('\n')
    subdb.pop(0)
    # ON CONFLICT : 데이터가 중복되면 INSERT하지않음
    cur.execute("INSERT into subaccdata(acctime, content) VALUES (%s, %s) ON CONFLICT (acctime, content) DO NOTHING", subdb)

# db commit
conn.commit()   

In [8]:
# 오늘날짜의 지하철 이슈 찾아보기
today = date.today().isoformat() + '%'

# 조회해서 리스트(안에는 튜플)에 담기
cur.execute(f"SELECT * FROM subaccdata WHERE acctime LIKE '{today}' order by acctime ASC")
result_all = cur.fetchall()

sbstr=""
for i in result_all:
    for j in i:
        sbstr = sbstr + j + "\n"
    sbstr = sbstr + "\n"

print(sbstr[:-2])

2022-11-24 06:00
서울교통공사에서 알려드립니다. 금일 08시부터 4호선에서「전국장애인차별철폐연대」의 ‘장애인 권리예산 확보’를 위한 출근길 지하철 타기 선전전이 예정되어 있습니다. 이로 인해 4호선 해당구간 열차운행이 상당시간 지연될 수 있으니 이점 참고하여 열차를 이용해 주시기 바랍니다.
